<a href="https://colab.research.google.com/github/alexeynick/msu_oil_gas/blob/main/module2_lesson1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Задачи модуля №2

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt 
from plotly.subplots import make_subplots

import locale

locale.setlocale(locale.LC_ALL, '')
locale._override_localeconv = {'mon_thousands_sep': '_'}

In [ ]:
pd.set_option('display.max_colwidth', 100)

## Задача 2.1 Расчет запасов нефти и газа объемным методом

In [ ]:
data = {
    'Параметры': ['Площадь нефтеносности залежи',
                  'Эффективная нефтенасыщенная толщина залежи',
                  'Коэффициент пористости',
                  'Коэффициент нефтенасыщенности',
                  'Плотность нефти в поверхностных условиях',
                  'Переводной коэффициент из пластовых условий в поверхностные',
                  'Газовый фактор',
                  'Коэффициент нефтеотдачи',
                  'Добыча нефти с начала разработки на анализируемую дату',
                  ],
        'Единицы измерения': ['м2',
                              'м',
                              'д.ед.',
                              'д.ед.',
                              'т/м3',
                              'д.ед.',
                              'м3/т',
                              'д.ед.',
                              'тыс.т.',
                              ],
         'Значения': [6012,
                     16,
                     0.17,
                     0.515,
                     0.826,
                     0.915,
                     38.7,
                     0.585,
                     2016.9,
                     ]
        }

df = pd.DataFrame(
    data, 
    columns = ['Параметры','Единицы измерения', 'Значения'],
    index=[1, 2, 3, 4, 5, 6, 7, 8, 9])

df

,Параметры,Единицы измерения,Значения
1,Площадь нефтеносности залежи,м2,6012.000
2,Эффективная нефтенасыщенная толщина залежи,м,16.000
3,Коэффициент пористости,д.ед.,0.170
4,Коэффициент нефтенасыщенности,д.ед.,0.515
5,Плотность нефти в поверхностных условиях,т/м3,0.826
6,Переводной коэффициент из пластовых условий в поверхностные,д.ед.,0.915
7,Газовый фактор,м3/т,38.700
8,Коэффициент нефтеотдачи,д.ед.,0.585
9,Добыча нефти с начала разработки на анализируемую дату,тыс.т.,2016.900


$$Q_{ИЗВ} = F \cdot h_{НН} \cdot m \cdot K _{НН} \cdot \rho_{Н} \cdot \beta \cdot \eta $$

где:
- $Q_{ИЗВ}$ - извлекаемые (промышленные) запасы нефти, $т$;
- $F$ — площадь нефтеносности залежи, $м^{2}$;
- $h_{НН}$ — эффективная нефтенасыщенная толщина залежи, $м$;
- $m$ — коэффициент открытой пористости нефтесодержащих пород;
- $K _{НН}$ — коэффициент насыщения пласта нефтью (коэффициент нефтенасыщенности);
- $\rho_{Н}$ — плотность нефти в поверхностных условиях, $т/м^{3}$;
- $\beta$ — пересчетный коэффициент, учитывающий усадку нефти $\beta = \cfrac{1}{b}$ (${b}$ - объемный коэффициент пластовой нефти);
- $\eta$ — коэффициент нефтеотдачи (КИН);


$K _{НН}$ - Коэффициент нефтенасыщенности или газонасыщенности (oil or gas saturation factor) - отношение объема нефти (газа), содержащейся в порах (пустотах) пласта, к общему объему всех пор (пустот) нефтеносного (газоносного) пласта в пластовых условиях (М.А. Жданов, 1962; Ф.И. Котяхов, И.Х. Абрикосов, И.С. Гутман, 1970). 

$m$ - в нефтегазопромысловой геологии более важен коэффициент открытой пористости, т.к. он характеризует объем углеводородов, содержащийся в породе. На практике коэффициент открытой пористости определяется в лабораторных условиях по методу Преображенского или по данным геофизических исследований в скважине (ГИС).

Полная пористость – это объем всех пор, находящихся в горной породе.

Открытая пористость – это объем пор, сообщающихся между  cобой. Количественно та и другая пористость выражается коэффициентом пористости, который представляет собой отношение объема пор к объему образца породы


Рассчитать:

- 1) Начальные геологические запасы нефти
- 2) Начальные извлекаемые запасы нефти
- 3) Остаточные геологические запасы нефти
- 4) Остаточные извлекаемые запасы нефти
- 5) Начальные геологические запасы газа
- 6) Начальные извлекаемые запасы газа
- 7) Остаточные балансовые запасы газа
- 8) Остаточные извлекаемые запасы газа

### Расчеты

$Q_{ИЗВ} = Q_{ГЕОЛ} \cdot \eta $

$Q_{ГЕОЛ} = F \cdot h_{НН} \cdot m \cdot K _{НН} \cdot \rho_{Н} \cdot \beta $

$Q_{ОСТ\,ГЕОЛ} = Q_{ГЕОЛ} - Q_{ДОБ} $

$Q_{ОСТ\,ИЗВ} = Q_{ИЗВ} - Q_{ДОБ} $




In [ ]:
values = df['Значения']

In [ ]:
F_value = values[1]
hnn_value = values[2]
m_value = values[3]
Knn_value = values[4]
rhon_value = values[5]
beta_value = values[6]
eta_value = values[8]
q_dob = values[9]

q_geol = F_value * hnn_value * m_value * Knn_value * rhon_value * beta_value
q_ost_geol = q_geol * eta_value

In [ ]:
print('Начальные геологические запасы нефти: {0:,.0f}'.format(q_geol))
print('Начальные извлекаемые запасы нефти: {0:,.0f}'.format(q_ost_geol))
print('Остаточные геологические запасы нефти: {0:,.0f}'.format(q_geol - q_dob))
print('Остаточные извлекаемые запасы нефти: {0:,.0f}'.format(q_ost_geol - q_dob))

Начальные геологические запасы нефти: 6,365
Начальные извлекаемые запасы нефти: 3,724
Остаточные геологические запасы нефти: 4,348
Остаточные извлекаемые запасы нефти: 1,707


$V_{ГЕОЛ} = Q_{ГЕОЛ} \cdot Г $

$V_{ИЗВ} = Q_{ИЗВ} \cdot Г $

$V_{ОСТ\,ГЕОЛ} = Q_{ОСТ\,ГЕОЛ} \cdot Г $

$V_{ОСТ\,ИЗВ} = Q_{ОСТ\,ИЗВ} \cdot Г $

где:
- $Г$ - газовый фактор (объёмное содержание газа в единице массы нефти), $м^{3}/т$

In [ ]:
gfac_value = values[7]

print('Начальные геологические запасы газа: {0:,.0f}'.format(q_geol * gfac_value))
print('Начальные извлекаемые запасы газа: {0:,.0f}'.format(q_ost_geol * gfac_value))
print('Остаточные геологические запасы газа: {0:,.0f}'.format((q_geol - q_dob) * gfac_value))
print('Остаточные извлекаемые запасы газа: {0:,.0f}'.format((q_ost_geol - q_dob) * gfac_value))

Начальные геологические запасы газа: 246,324
Начальные извлекаемые запасы газа: 144,100
Остаточные геологические запасы газа: 168,270
Остаточные извлекаемые запасы газа: 66,046


## Задача 2.2 Определение количества нефти, которое возможно добыть при упругом режиме пласта (свободное фонтанирование)

Определить количество нефти, полученное за счет упругого расширения нефти, воды и горной породы, если:
- площадь области нефтеносности $S_{Н}=900\,га$
- законтурная вода занимает площадь $S_{В}=10\,000\,га$
- средняя толщина пласта $h=15\,м$
- пористость пласта $m=0,17$
- водонасыщенность в зоне нефтеносности $\sigma_{В}=20\% $
- коэффициент сжимаемости нефти $\beta_{Н}=6.12 \cdot 10^{-10} м2/Н$
- коэффициент сжимаемости воды $\beta_{В}=4.28 \cdot 10^{-10} м2/Н$
- коэффициент сжимаемости породы $\beta_{С}=2.04 \cdot 10^{-10} м2/Н$. 

Пластовое давление снижается от 30 до 20 МПа.

Также определить коэффициент извлечения нефти (КИН).

Рассчитать:

- 1) Начальный объем нефти
- 2) Объем нефти, вытесняемый из зоны нефтеносности при падении давления за счет сжимаемости нефти и пористой среды
- 3) Объем вытсненной нефти в зоне нефтеносности при падении давления за счет расширения воды и породы в зоне нефтеносности
- 4) Объем нефти, вытесняемый из окружающей зоны водоносности при падении давления за счет упругости воды и пласта
- 5) Рассчитать КИН

### Расчеты

$Q_{ГЕОЛ} = S_{Н} \cdot h \cdot (1 - \sigma_{В}) \cdot m $


$V_{1} = S_{Н} \cdot h \cdot (1 - \sigma_{В}) \cdot m \cdot \beta_{Н} \cdot \Delta P + S_{Н} \cdot h \cdot (1 - \sigma_{В}) \cdot \beta_{С} \cdot \Delta P = $


$=S_{Н} \cdot h \cdot (1 - \sigma_{В}) \cdot (\beta_{Н} \cdot m + \beta_{С}) \cdot \Delta P$


$V_{2} = S_{Н} \cdot h \cdot \sigma_{В} \cdot m \cdot \beta_{В} \cdot \Delta P + S_{Н} \cdot h \cdot \sigma_{В} \cdot \beta_{С} \cdot \Delta P = $


$=S_{Н} \cdot h \cdot \sigma_{В} \cdot (\beta_{В} \cdot m + \beta_{С}) \cdot \Delta P$


$V_{3} = S_{В} \cdot h \cdot m \cdot \beta_{В} \cdot \Delta P + S_{Н} \cdot h \cdot \beta_{С} \cdot \Delta P = $


$=S_{Н} \cdot h \cdot (\beta_{В} \cdot m + \beta_{С}) \cdot \Delta P$


$КИН = \cfrac{V_{1} + V_{2} + V_{3}}{Q_{ГЕОЛ}} $




In [ ]:
HA_TO_SQRTM = 10000

In [ ]:
sn_value = 900
sv_value = 10000
hnn_value = 15
m_value = 0.17
sigmav_value = 0.2
betan_value = 6.12 * 10**-10
betav_value = 4.28 * 10**-10
betas_value = 2.04 * 10**-10
p1 = 30 * 10**6
p2 = 20 * 10**6

q_geol = sn_value * HA_TO_SQRTM * hnn_value * (1 - sigmav_value) * m_value
v_1 = sn_value * HA_TO_SQRTM * hnn_value * (1 - sigmav_value) * (betan_value * m_value + betas_value) * (p1 - p2)
v_2 = sn_value * HA_TO_SQRTM * hnn_value * sigmav_value * (betav_value * m_value + betas_value) * (p1 - p2)
v_3 = sv_value * HA_TO_SQRTM * hnn_value * (betav_value * m_value + betas_value) * (p1 - p2)

In [ ]:
print('Начальный объем нефти: {0:,.0f}'.format(q_geol))
print('Объем нефти, вытесняемый из зоны нефтеносности при падении давления за счет сжимаемости нефти и пористой среды: {0:,.0f}'.format(v_1))
print('Объем вытсненной нефти в зоне нефтеносности при падении давления за счет расширения воды и породы в зоне нефтеносности: {0:,.0f}'.format(v_2))
print('Объем нефти, вытесняемый из окружающей зоны водоносности при падении давления за счет упругости воды и пласта: {0:,.0f}'.format(v_3))
print('КИН: {0:,.2%}'.format((v_1 + v_2 + v_3) / q_geol))

Начальный объем нефти: 18,360,000
Объем нефти, вытесняемый из зоны нефтеносности при падении давления за счет сжимаемости нефти и пористой среды: 332,683
Объем вытсненной нефти в зоне нефтеносности при падении давления за счет расширения воды и породы в зоне нефтеносности: 74,725
Объем нефти, вытесняемый из окружающей зоны водоносности при падении давления за счет упругости воды и пласта: 4,151,400
КИН: 24.83%


## Задача 2.3 Расчет запасов газа по методу падения давления

$Q_{ИЗВ} = \cfrac{(Q_{2} - Q_{1}) \cdot (P_{1} \cdot \alpha_{1} - P_{K} \cdot \alpha_{K})}{(P_{1} \cdot \alpha_{1} - P_{2} \cdot \alpha_{2})} 
 $

где:
- $Q_{ИЗВ}$ - извлекаемые запасы газа, $млрд\,м^{3}$;
- $Q_{1}$ — количество газа, добытого с начала разработки на 1-ю дату замера, $млрд\,м^{3}$;
- $Q_{2}$ — количество газа, добытого с начала разработки на 2-ю дату замера, $млрд\,м^{3}$;
- $\alpha_{1}$ — поправка за отклонение от идеального газа на 1-ю дату замера;
- $\alpha_{2}$ — поправка за отклонение от идеального газа на 2-ю дату замера;
- $\alpha_{K}$ — поправка за отклонение от идеального газа (конечные условия);
- $P_{1}$ — пластовое давление на 1-ю дату замера, $МПа$;
- $P_{2}$ — пластовое давление на 2-ю дату замера, $МПа$;
- $P_{K}$ — конечное пластовое давление, $МПа$;

In [ ]:
q1_value = 10
q2_value = 100
p1_value = 28
p2_value = 12
pk_value = 1
alpha1_value = 1 / 0.9828
alpha2_value = 1 / 0.9703
alphak_value = 1 / 0.9946


In [ ]:
ATM_TO_PA = 101325

In [ ]:
q_geol = (
    (q2_value - q1_value) 
    * (p1_value * ATM_TO_PA * alpha1_value - pk_value * ATM_TO_PA * alphak_value)
    / (p1_value * ATM_TO_PA * alpha1_value - p2_value * ATM_TO_PA * alpha2_value)
)

In [ ]:
print('Извлекаемые запасы газа: {0:,.2f} млрд. куб. м'.format(q_geol))

Извлекаемые запасы газа: 153.42 млрд. куб. м


## Задача 2.4 Определение необходимого числа скважин в условиях максимальной добычи извлекаемых запасов 

При проектировании разработки нефтяного месторождения в нем было выделено два объекта разработки. Пласты характеризуются неоднородным строением и содержат много пропластков и линз. 

В результате лабораторного изучения процесса вытеснения нефти из пород-коллекторов определены коэффициенты вытеснения $K_{ВЫТ1}=0,7$ и $K_{ВЫТ2}=0,6$ для первого и второго объектов, а также получены зависимости коэффициентов охвата первого и второго объектов воздействием $K_{ОХВ1}$, $K_{ОХВ2}$ от соответствующих параметров плотности сеток скважин $S_{C1}$, $S_{C2}$, они имеют вид:
- $K_{ОХВ1}=1-0,055 \cdot S_{C1}$
- $K_{ОХВ2}=1-0,00833 \cdot S_{C2}$

Геологические запасы $Q_{Г1}=80$ млн т, $Q_{Г2}=30$ млн т. Площадь нефтеносности $S_{1}=5000$ га, $S_{2}=1200$ га.
На оба объекта решено пробурить n=150 скважин.

Найти какое число скважин следует пробурить на каждый объект разработки с тем, чтобы суммарные извлекаемые запасы для месторождения в целом получились максимальными.

$КИН = \cfrac{Q_{ИЗВ}}{Q_{ГЕОЛ}}$

$КИН = K_{ВЫТ} \cdot K_{ОХВ} \cdot K_{ЗАВ}$

где:

- $K_{ВЫТ}$ - Коэффициент вытеснения нефти водой - отношение извлеченной из пласта нефти к ее запасам, первоначально находившимся в части пласта, подверженной воздействию заводнением. Определяется лабораторно на образцах породы.
- $K_{ОХВ}$ - Коэффициент охвата - отношение запасов нефти, первоначально находившихся в части пласта, подверженной воздействию заводнением, к геологическим запасам нефти в пласте
- $K_{ЗАВ}$ - Коэффициент заводнения - это отношение накопленной добычи нефти к дренируемым запасам. Характеризует ту часть дренируемых запасов, которую возможно добыть.

### Расчеты

$n=n_{1} + n_{2}$

$S_{С1} = \cfrac{S_{1} } {n_{1}}, S_{С2} = \cfrac{S_{2} } {n_{2}} $

$Q_{ИЗВ1} = Q_{Г1} \cdot КИН_{1} = Q_{Г1} \cdot K_{ВЫТ1} \cdot K_{ОХВ1} = Q_{Г1} \cdot K_{ВЫТ1} \cdot (1 - 0.055 \cdot S_{C1})$

$Q_{ИЗВ2} = Q_{Г2} \cdot КИН_{2} = Q_{Г2} \cdot K_{ВЫТ2} \cdot K_{ОХВ2} = Q_{Г2} \cdot K_{ВЫТ2} \cdot (1 - 0.00833 \cdot S_{C2})$

$Q_{\Sigma} = Q_{ИЗВ1} + Q_{ИЗВ2} = $

$ = Q_{Г1} \cdot K_{ВЫТ1} + Q_{Г2} \cdot K_{ВЫТ2} - \cfrac{0.055 \cdot Q_{Г1} \cdot K_{ВЫТ1} \cdot S_{1}}{n_{1}} - \cfrac{0.00833 \cdot Q_{Г2} \cdot K_{ВЫТ2} \cdot S_{2}}{n - n_{1}} = $

$ = A - \cfrac{B}{n_{1}} - \cfrac{C}{n-n_{1}}$

Найдем локальный экстремум $Q_{\Sigma}$ как функции от $n_{1}$

$Q_{\Sigma}^{'} = 0 $

$\cfrac{dQ_{\Sigma}}{dn} = \cfrac{B}{n_{1}^{2}} - \cfrac{C}{(n-n_{1})^{2}} = 0$

$(B-C) \cdot n_{1}^{2} - 2Bn \cdot n_{1} + Bn^{2}= 0$


Формула нахождения корней квадратных уравнений с помощью дискриминанта:

$a \cdot x^2 + b \cdot x + c = 0$

$x_{1,2} = \cfrac{-b\pm\sqrt{D}}{2a}$,

где

$D = b^2 - 4ac$

Соответственно:

$D = 4B^2n^2 - 4Bn^2(B-C) = 4B^2n^2 - 4B^2n^2 + 4BCn^2$

$\sqrt{D} = 2n \cdot \sqrt{B} \cdot \sqrt{C}$

$n_{1,2} = \cfrac{2Bn\pm2n \cdot \sqrt{B} \cdot \sqrt{C}}{2(B-C)} = n \cdot \cfrac{B\pm\sqrt{B} \cdot \sqrt{C}}{(B-C)}$

$n_{1} = \cfrac{n\sqrt{B}}{\sqrt{B}-\sqrt{C}}$

$n_{2} = \cfrac{n\sqrt{B}}{\sqrt{B}+\sqrt{C}}$

In [ ]:
s1_value = 5000
s2_value = 1200
q1_geol = 80
q2_geol = 30
kvyt1_value = 0.7
kvyt2_value = 0.6

B_value = 0.055 * q1_geol * kvyt1_value * s1_value
C_value = 0.00833 * q2_geol * kvyt2_value * s2_value

In [ ]:
B_value ** 0.5

124.09673645990857

In [ ]:
C_value ** 0.5

13.413724315043902

In [ ]:
150*(B_value ** 0.5) / (B_value ** 0.5 - C_value ** 0.5)

168.17856786028875

Подходящее решение:

In [ ]:
150*(B_value ** 0.5) / (B_value ** 0.5 + C_value ** 0.5)

135.367959383472

Соответственно:

$n_{1} = 135$

$n_{2} = 15$

## Задача 2.5 Определение параметров по методу материального баланса

Определить текущую нефтенасыщенность $S_{Н}$ нефтяной залежи на различные моменты времени, когда пластовое давление в процессе разработки при режиме растворенного газа $P_1 = 21.0\;МПа, P_2=18.4\;МПа, P_3=17\;МПа$. 

Основные данные о параметрах нефти и газа приведены в таблице. 
Давление насыщения $P_Н = 22\;МПа$. Насыщенность порового объема связанной водой $S_{СВ} = 0.12$.

### Решение

При упругом режиме параметр нефтенасыщенности постоянен $S_{Н}=Const$

В нашем случае пластовое давление опустилось ниже давления насыщения, соответственно добыча идет в режиме растворенного газа. Необходимо определить $S_{Н}$ в различные моменты времени.

В пластовых условиях (при пластовых давлениях) газ находится в растворённом состоянии и только при снижении давления начинает выделяться из нефти. 
Количество растворённого в нефти газа характеризуется понятием газовый фактор (Г). 
Этот показатель используется при разработке нефтяных и газонефтяных месторождений.
Газовый фактор представляет собой объёмное содержание газа в единице массы нефти $Г = \cfrac{Q_Г}{Q_Н}$, единица измерения - $м^3/т$.

Заданные параметры:

- $b_{Н}$ - объемный коэффициент нефти
- $b_{Н0}$ - объемный коэффициент нефти при давлении насыщения
- $b_{Г}$ - объемный коэффициент газа
- $Г_0$ - газовый фактор при давлении насыщения
- $Г$ - текущий газовый фактор за весь период добычи
- $\overlineГ$ - усредненный газовый фактор


$КИН = \cfrac{Q_{НАК}}{Q_{ГЕОЛ}}$

${Q_{ГЕОЛ}} = Q_{НАК}\cdot \cfrac{(b_{Н} + (\overlineГ - Г) \cdot b_{Г})}{b_{Н} - b_{Н0} + (Г_0 - Г) \cdot b_{Г}}$

$S_{Н}=(1 - КИН) \cdot \cfrac{b_{Н}}{b_{Н0}} \cdot (1 - S_{СВ}) = (1 - \cfrac{b_{Н} - b_{Н0} + (Г_0 - Г) \cdot b_{Г}}{(b_{Н} + (\overlineГ - Г) \cdot b_{Г})}) \cdot \cfrac{b_{Н}}{b_{Н0}} \cdot (1 - S_{СВ})$

In [ ]:
b_n0 = 1.5
g_0 = 180
s_w = 0.12

df_data = pd.DataFrame(
    { 'p' : [21, 18.4, 17],
     'b_n' : [1.481, 1.44, 1.42],
     'b_g' : [0.009, 0.0075, 0.0067],
     'g_cur' : [100, 110, 120],
     'g_mid' : [220, 290, 330],
     }
)

In [ ]:
df_data['kin'] = np.round(
    (df_data['b_n'] - b_n0 + (g_0 - df_data['g_cur']) * df_data['b_g'] ) / 
    (df_data['b_n'] + (df_data['g_mid'] - df_data['g_cur']) * df_data['b_g'])
    , 3
)

df_data['s_n'] = np.round(
    (1 - df_data['kin']) * df_data['b_n'] / b_n0 * (1 - s_w)
    , 2
)

In [ ]:
df_data

,p,b_n,b_g,g_cur,g_mid,kin,s_n
0,21.0,1.481,0.0090,100,220,0.274,0.63
1,18.4,1.440,0.0075,110,290,0.167,0.70
2,17.0,1.420,0.0067,120,330,0.114,0.74


In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_data['p'], y=df_data['kin'],
                    mode='lines+markers',
                    name='КИН'))

fig.add_trace(go.Scatter(x=df_data['p'], y=df_data['s_n'],
                    mode='lines+markers',
                    name='Нефтенасыщенность'))

fig.update_layout( 
    xaxis=dict(
        domain=[0.2, 0.9],
        tickmode = 'array',
        tickvals = df_data['p'],
        title='Давление (по времени)',
        autorange = "reversed"
    ),
    title={
        'text': "Динамика параметров при режиме растворенного газа",
        'y':0.95,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'},
    autosize=False,
    width=1000,
    height=500
)

fig.show()

## Задача 2.7 Прогнозирование показателей разработки месторождения и оценка эффективности использования пластовой энергии

Нефтяная залежь имеет газовую шапку, окружена активной пластовой водой и характеризуется сложным проявлением различных режимов работы продуктивного пласта.

Дано:
- $P_0 = P_{НАС}$ - начальное пластовое давление равно давлению насыщения
- $\alpha_0 = 0.25$ - начальная доля объема газовой шапки по отношению ко всему объему залежи в пределах контура нефтегазоносности

По результатам лабораторных исследований:
- $Г_0=150\;м^3/м^3$ - начальное газосодержание нефти
- начальный объемный коэффициент:
  - $b_{Г0} = 0.006$ - газа
  - $b_{Н0} = 1.475$ - нефти
- $S_{СВ}=0.12$ - насыщенность порового объема связанной водой

Среднее пластовое давление за определенный период эксплуатации снизилось до пластового давления, при котором:
- $Г_0=125\;м^3/м^3$ - газосодержание нефти
- объемный коэффициент:
  - $b_{Г} = 0.0063$ - газа
  - $b_{Н} = 1.415$ - нефти
  - $b_{В} = 1.028$ - воды

За этот период было добыто:
- $Q_{Н}=1.06 \cdot 10^6\;м^3$ - нефти (безводной)
- $V_{Г}=175 \cdot 10^6\;м^3$ - газа
- $V_{В}=5 \cdot 10^4\;м^3$ - воды

Причем, количество законтурной воды, внедрившейся в залежь:
- $W=1.2 \cdot 10^6\;м^3$

**Определить**:
    
    1. Начальные геологические запасы нефти $G_{Н}$
    2. Текущий КИН
    3. Текущую нефтенасыщенность 
    4. Относительную эффективность отдельных видов энергии (газовая шапка, растворенный газ, активная вода)

Решение:

1. Определяем долю объема газовой шапки от объема начальных геологических запасов нефти в залежи

$\alpha = \cfrac{\alpha_0}{1-\alpha_0}$

2. Определяем значение среднего за рассматриваемый период газового фактора

$Г^* = \cfrac{V_{Г}}{Q_{Н}}$

3. Составляем уравнение материального баланса в залежи на момент времени, когда пластовое давление снизилось до величины $P_{ПЛ}$ и добыто количество газа $V_{Г}$

$\Delta V_{Г} = G_{ГШ} + G_{ГР} = (\alpha \cdot G_{Н} \cdot \cfrac{b_{Н0}}{b_{Г0}} + G_{Н} \cdot Г_0 - V_{Г}) \cdot b_{Г} $

где 

$\Delta V_{Г}$ - оставшийся в залежи, приведенный к текущему пластовому давлению объем газа (в формуле выше - объем газа в газовой шапке + объем растворенного в нефтяном пласте газа - объем добытого газа), который в свою очередь состоит из:

- $G_{ГР}$ - растворенного в нефти газа:
  - $G_{ГР} = (G_{Н} - Q_{Н}) \cdot Г \cdot b_{Г}$

- $G_{ГШ}$ - газовой шапки (свободного газа):
  - $G_{ГШ} = (\alpha \cdot G_{Н} \cdot b_{Н0} + G_{Н} \cdot b_{Н0}) - (G_{Н} - Q_{Н}) \cdot b_{Н} - (W - V_{В} \cdot b_{В})$

4. Решаем уравнение материального баланса относительно $G_{Н}$

$$G_{Н} = \cfrac{Q_{Н} \cdot (b_{Н} + b_{Г}(Г^* - Г)) - (W - V_{В} \cdot b_{В})}{\alpha \cdot b_{Н0} (\cfrac{b_{Г}}{b_{Г0}} - 1) + b_{Н} - b_{Н0} + b_{Г}(Г_0 - Г)}$$

5. За рассматриваемый период коэффициент нефтеотдачи составил:

$$КИН = \cfrac{Q_{Н}}{G_{Н}}$$

6. Определим значение текущей нефтенасыщенности на конец рассматриваемого периода:

$$S_{Н}= \cfrac{(G_{Н} - Q_{Н}) \cdot b_{Н}}{G_{Н} \cdot b_{Н0}} \cdot (1 - S_{СВ}) = (1 - КИН) \cdot \cfrac{b_{Н}}{b_{Н0}} \cdot (1 - S_{СВ}) $$

7. Обратим внимание на то, что в уравнении материального баланса первое слагаемое в числителе выражает общее количество добытых нефти и газа, приведенное к пластовым условиям при текущем пластовом давлении:

$Q_{НГ} = Q_{Н} \cdot (b_{Н} + b_{Г}(Г^* - Г))$

Разделим левую и правую часть уравнения материального баланса на $Q_{НГ}$:

$$G_{Н} \cdot \cfrac{b_{Н} - b_{Н0} + b_{Г}(Г_0 - Г)}{Q_{НГ}} + G_{Н} \cdot \cfrac{\alpha \cdot b_{Н0} (\cfrac{b_{Г}}{b_{Г0}} - 1)}{Q_{НГ}} = 1 - \cfrac{W - V_{В} \cdot b_{В}}{Q_{НГ}}$$

или

$$J_{РГ} + J_{ГШ} + J_{В} = 1$$

где:
- $J_{РГ}$ - доля участия растворенного газа в вытеснении нефти
- $J_{ГШ}$ - доля участия газовой шапки в вытеснении нефти
- $J_{В}$ - доля участия свободной воды в вытеснении нефти



In [ ]:
alpha = 1/3
G_0 = 150
G = 125


b_g0 = 6 * 10 ** (-3)
b_n0 = 1.475
s_v = 0.12

b_g = 6.3 * 10 ** (-3)
b_n = 1.415
b_v = 1.028

Q = 1.06 * (10 ** 6)
V_g = 175 * (10 ** 6)
V_v = 5 * (10 ** 4)
W = 1.2 * (10 ** 6)

G_mid = V_g / Q

In [ ]:
Q_ng = Q * (b_n + b_g * (G_mid - G))
g_n = (
    (Q_ng - (W - V_v * b_v)) /
    (alpha * b_n0 * (b_g / b_g0 - 1) + b_n - b_n0 + b_g * (G_0 - G))
)
g_n

5070716.723549487

In [ ]:
kin = Q / g_n
kin

0.20904342675605098

In [ ]:
s_n = (1 - kin) * (b_n / b_n0) * (1 - s_v)
s_n

0.6677282203412646

In [ ]:
print('КПД растворенного газа: {:.2f}'.format((g_n / Q_ng) * (b_n - b_n0 + b_g * (G_0 - G)) ))
print('КПД газовой шапки: {:.2f}'.format((g_n / Q_ng) * (alpha * b_n0 * (b_g / b_g0 - 1)) ))
print('КПД воды: {:.2f}'.format( (W - V_v * b_v) / Q_ng ))

КПД растворенного газа: 0.28
КПД газовой шапки: 0.07
КПД воды: 0.65


## Задача 2.8 Экспресс расчет технологических показателей разработки нефтяных месторождений с заводнением

Некоторое вновь открытое нефтяное месторождение с геологическими запасами $G=180$ млн.т. нефти вводится в разработке. Определена зависимость текущей обводненности добываемой продукции $\nu$ от текущей нефтеотдачи (КИН):

$$
\begin{equation}
    \begin{matrix}
    \nu (\eta) =
    \left\{
    \begin{matrix}
    34.8\cdot(\eta-0.1)^2 & 0.1 < \eta \leq 0.25 \\
    1.232\cdot(\eta-0.1)^{1/4} & 0.25 < \eta \leq 0.5 \\
    \end{matrix} \right.
    \end{matrix}
\end{equation}
$$

Требуется осуществить в кратчайший срок оценку возможной добычи нефти и воды из 
месторождения в ближайшие 15 лет. Темп ввода скважин в эксплуатацию на данном 
месторождении предполагается сохранить таким же, как и на месторождение-аналоге 
(сохраняется зависимость $\nu=\nu(\eta)$).

Изменение годового отбора жидкости во времени приведено в таблице:

In [ ]:
def calculate_nu_from_kin(kin):
  nu = 0
  if kin > 0.1 and kin <= 0.25:
    nu = 34.8 * (kin - 0.1)**2
  elif kin > 0.25 and kin <= 0.5:
    nu = 1.232 * (kin - 0.1)**0.25

  return np.round(nu, 3)

In [ ]:
total_years = 15

df_data = pd.DataFrame(
    { 'year' : range(0, total_years + 1),
     'q_fl_year' : [0.0, 0.9, 1.6, 2.6, 3.5] + list(np.full(total_years - 4, 5))
     }
)

In [ ]:
df_nu_kin = pd.DataFrame(
    { 'kin' : np.arange(0.1, 0.51, 0.01),
     }
)

df_nu_kin['nu'] = df_nu_kin.apply(lambda row: calculate_nu_from_kin(row['kin']), axis=1)

In [ ]:
fig = make_subplots(
    subplot_titles=[
                    'График годового отбора жидкости по годам',
                    'Зависимость текущей обводненности <br>от текущей нефтеотдачи'],
    rows=1, 
    cols=2)


fig.add_trace(go.Scatter(x=df_data['year'], y=df_data['q_fl_year'],
                    mode='lines+markers',
                    name='Годовой отбор жидкости, 10^6'),
    row=1, col=1)

fig.add_trace(go.Scatter(x=df_nu_kin['kin'], y=df_nu_kin['nu'],
                    mode='lines+markers',
                    name='Обводненность, д.ед.'),
    row=1, col=2)

fig['layout']['xaxis1']['title']='Год'
fig['layout']['xaxis1']['tickangle']=-45
fig['layout']['xaxis1']['tickmode']='array'
fig['layout']['xaxis1']['tickvals']=df_data['year']
fig['layout']['yaxis1']['title']='Годовой отбор жидкости, 10^6'

fig['layout']['xaxis2']['title']='КИН'
fig['layout']['xaxis2']['tickangle']=-45
fig['layout']['yaxis2']['title']='Обводненность, д.ед.'



fig.update_layout( 
    title={
        'text': "Исходные данные",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    autosize=False,
    width=1400,
    height=700)


fig.show()

In [ ]:
df_result = pd.concat([df_data, df_nu_kin[:(total_years + 1)]], axis=1)
df_result.head(5)

,year,q_fl_year,kin,nu
0,0,0.0,0.10,0.000
1,1,0.9,0.11,0.003
2,2,1.6,0.12,0.014
3,3,2.6,0.13,0.031
4,4,3.5,0.14,0.056


In [ ]:
df_result['q_fl_total'] = df_result['q_fl_year'].cumsum()
df_result['q_oil_year'] = df_result['q_fl_year'] * (1 - df_result['nu'])
df_result['q_oil_total'] = df_result['q_oil_year'].cumsum()
df_result.tail(5)

,year,q_fl_year,kin,nu,q_fl_total,q_oil_year,q_oil_total
11,11,5.0,0.21,0.421,43.6,2.895,35.0133
12,12,5.0,0.22,0.501,48.6,2.495,37.5083
13,13,5.0,0.23,0.588,53.6,2.060,39.5683
14,14,5.0,0.24,0.682,58.6,1.590,41.1583
15,15,5.0,0.25,0.783,63.6,1.085,42.2433


In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_result['year'], y=df_result['q_fl_year'],
                    mode='lines+markers',
                    name='Годовой отбор жидкости, 10^6'))

fig.add_trace(go.Scatter(x=df_result['year'], y=df_result['q_fl_total'],
                    mode='lines+markers',
                    name='Накопленный объем жидкости, 10^6',
    yaxis="y2"))

fig.add_trace(go.Scatter(x=df_result['year'], y=df_result['q_oil_total'],
                    mode='lines+markers',
                    name='Накопленный объем нефти, 10^6',
    yaxis="y2"))

fig.update_layout( 
    xaxis=dict(
        domain=[0.2, 0.9],
        tickmode = 'array',
        tickvals = df_data['year'],
        tickangle=-45,
        title='Год',
    ),

    yaxis=dict(
        range=[0, 7],
        title="Годовой отбор жидкости, млн т",
        titlefont=dict(
            color="#1f77b4"
        ),
        tickfont=dict(
            color="#1f77b4"
        )
    ),

    yaxis2=dict(
        range=[0, 70],
        title="Накопленные объемы, млн т",
        titlefont=dict(
            color="#ff7f0e"
        ),
        tickfont=dict(
            color="#ff7f0e"
        ),
        anchor="x",
        overlaying="y",
        side="right"
    ),

    title={
        'text': "Объемы (накопленные объемы) по годам",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    #title='Объемы (накопленные объемы) по годам',
    autosize=False,
    width=1500,
    height=700)


fig.show()

## Задача 2.11 Закон Дарси

Горизонтальная цилиндрическая труба заполнена песком. Через эту трубу фильтруется жидкость. Даны внутренний диаметр $d=10\;см$ и длина трубы $L=45\;cм$, пористость $m=0.18$, перепад давления $\Delta P=18*10^4 Па$, вязкость $\mu =3\;сПз$ (сантипуаз = $10^{-3}\;Па/с$ = $1\;мПа/с$), плотность $\rho = 850\;кг/м^3$ и расход жидкости $Q=5\;см^3/с$. 

Определить: проницаемость песка $k$.

Решение:

$Q = w \cdot S$

$Q_{М} = Q \cdot \rho$

где:

- $w$ - скорость фильтрации;
- $S$ - площадь сечения

$w = \cfrac{k}{\mu} \cdot \cfrac{\Delta P}{L}$

$Q = \cfrac{k}{\mu} \cdot \cfrac{\Delta P}{L} \cdot S = \cfrac{k}{\mu} \cdot \cfrac{\Delta P}{L} \cdot \cfrac{\pi \cdot d^2}{4} $

$k = \cfrac{4Q \cdot \mu \cdot L}{\Delta P \cdot \pi \cdot d^2}$

In [ ]:
q = 5 * 10**(-6)
mu = 3 * 10 ** (-3)
l = 45 * 10**(-2)
delta_P = 18* (10**4)
d = 0.1

DARCI_TO_SQM = 0.9869 * 10 ** (-12)

In [ ]:
k = (4 * q * mu * l) / (delta_P * np.pi * d**2)
k / DARCI_TO_SQM

4.838026439109189

Ответ: ~4.84 Дарси

1 Дарси ~ $0.9869 \cdot 10^{−12} \;м^2$

## Задача 2.12 Закон Дарси

Определить дебит дренажной галереи шириной $В = 100\;м$, если толщина пласта $h=10\;м$, расстояние до контура питания $L=10\;км$, проницаемость пласта $k=1\;Д$, динамический коэффициент вязкости жидкости $\mu=1\;сПз$, давление на контуре питания $P_{К} =100\;атм$ и давление в галерее $P_{Г} =75\;атм$. 

Движение жидкости напорное по закону Дарси.

Решение:

Движение жидкости только по оси $X$

$Q = w_{X} \cdot S = \cfrac{k}{\mu} \cdot \cfrac{P_{К} - P_{Г}}{L} \cdot B \cdot h = \cfrac {k \cdot \ B \cdot h \cdot \Delta P}{\mu \cdot L}$

In [ ]:
k = 1 * DARCI_TO_SQM
mu = 1 * 10 ** (-3)
delta_P = 25 * ATM_TO_PA
l = 10 * 10 ** 3
b = 100
h = 10

SECOND_IN_DAY = 86400

In [ ]:
q = (k * b * h * delta_P) / (mu * l)
q * SECOND_IN_DAY

21.599490779999996

Ответ: ~21.6 $м^3/сут$

## Задача 2.13 Закон Дарси

Построить график распределения давления при одномерном движении в пласте несжимаемой жидкости по линейному закону фильтрации. Заданы длина $L=5\;км$, ширина $B=300\;м$ и толщина пласта $h=10\;м$, коэффициент проницаемости $k=0.8\;Д$, давление в галерее стока $P_{Г}=2.94\;МПа$ и ее дебит $Q=30\;м^3/сут$, динамическая вязкость жидкости $\mu=4\;сПз$.

Решение:

$Q = w_{X} \cdot S = \cfrac{k}{\mu} \cdot \cfrac{P_{К} - P_{Г}}{L} \cdot B \cdot h $

$P_{К} = P_{Г} + \cfrac{Q \cdot \mu \cdot L}{k \cdot B \cdot h}$

In [ ]:
q = 30 / SECOND_IN_DAY
mu = 4 * 10 ** (-3)
L = 5 * 10 ** 3
k = 0.8 * DARCI_TO_SQM
b = 300
h = 10
p_g = 2.94 * 10 ** 6

In [ ]:
p_k = p_g + (q * mu * l) / (k * b * h)
p_k

8803853.518124469

In [ ]:
p_k = p_k / 10 ** 6
p_g = p_g / 10 ** 6

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=[0, l / 10** 3], y=[p_k, p_g],
                    mode='text+lines+markers',
                    text=["P контура<br>{:.1f} МПа".format(p_k), 
                          "P галереи<br>{:.1f} МПа".format(p_g)],
                    textposition="top right",
                    name='КИН'))

fig.update_layout( 
    xaxis=dict(
        domain=[0.2, 0.9],
        range=[-2, 12],
        title='Расстояние (км)',
        zeroline=True,
        zerolinewidth=2, 
        zerolinecolor='red',
    ),

    yaxis=dict(
        range=[-1, 10],
        title="Давление, МПа",
        titlefont=dict(
            color="#1f77b4"
        ),
        tickfont=dict(
            color="#1f77b4"
        ),
        zeroline=True,
        zerolinewidth=2, 
        zerolinecolor='red'
    ),

    title={
        'text': "Распределение давления при одномерном движении в пласте несжимаемой жидкости",
        'y':0.95,
        'x':0.55,
        'xanchor': 'center',
        'yanchor': 'top'},
    autosize=False,
    width=1000,
    height=500
)

fig.show()

## Задача 2.14 Закон Дарси

Определить дебит нефтяной скважины (в т/сут) в случае установившейся плоско-радиальной фильтрации жидкости по закону Дарси, если известно, что давление на контуре питания $P_{К}=11\;МПа$, давление на забое скважины $P_{С}=7\;МПа$, проницаемость пласта $k=1\;Д$, толщина пласта $h=10\;м$, диаметр скважины $d_{С}=20\;см$, радиус контура питания $R_{К}=15\;км$, динамический коэффициент вязкости жидкости $\mu=4\;мПа*c$ и плотность $\rho=860\;кг/м^3$

Решение:

Формула Дюпюи (определение дебита центральной скважины в круговом пласте):

$Q = \cfrac{2 \cdot \pi \cdot k \cdot h}{\mu} \cdot \cfrac{P_{К} - P_{С}}{ln \cfrac{R_{К}}{r_{С}}}$

In [ ]:
k = 1 * DARCI_TO_SQM
h = 10
mu = 4 * 10 ** (-3)
delta_P = 4 * 10 ** 6
R = 15 * 10 ** 3
d = 20

In [ ]:
q = (2 * np.pi * k * h * delta_P) / (mu * np.log(2 * R / d))
q * SECOND_IN_DAY

732.5851290191985

Ответ: ~0.73 т/сут

## Задача 2.15 Скин фактор

Нефтяная скважина диаметром $d=20\;см$, вскрывшая пласт толщиной $h=10\;м$, проницаемостью $k=0.3\;Д$, дает дебит $Q=130\;м^3/сут$ при депрессии $\Delta P=2\;МПа$. Вязкость нефти $\mu=3\;мПа*с$. Расстояние до контура питания $R_{К}=700\;м$. Определить коэффициент, учитывающий дополнительное фильтрационное сопротивление за счет несовершенства скважины $S=S_1+S_2$, коэффициент совершенства скважины $\delta$ и приведенный радиус скважины $r_{c'}$


Решение:

$\delta = \cfrac{Q_{НЕС}}{Q_{СОВ}}$

$r_{c'} = r_{c} \cdot exp^{-S}$

Формула Дюпюи (определение дебита центральной скважины в круговом пласте):

$Q_{СОВ} = \cfrac{2 \cdot \pi \cdot k \cdot h}{\mu} \cdot \cfrac{P_{К} - P_{С}}{ln \cfrac{R_{К}}{r_{С}}}$

$Q_{НЕС} = \cfrac{2 \cdot \pi \cdot k \cdot h}{\mu} \cdot \cfrac{P_{К} - P_{С}}{ln \cfrac{R_{К}}{r_{С}} + S}$


Соответственно:

$\delta = \cfrac{Q_{НЕС}}{Q_{СОВ}} = \cfrac{ln \cfrac{R_{К}}{r_{С}}}{ln \cfrac{R_{К}}{r_{С}} + S}$ 

$Q_{НЕС} \cdot \mu \cdot (ln \cfrac{R_{К}}{r_{С}} + S) = 2 \cdot \pi \cdot k \cdot h \cdot \Delta P$

$S = \cfrac{2 \cdot \pi \cdot k \cdot h \cdot \Delta P}{Q_{НЕС} \cdot \mu} - ln \cfrac{R_{К}}{r_{С}}$

In [ ]:
k = 0.3 * DARCI_TO_SQM
h = 10
delta_P = 2 * 10 ** 6
q = 130 / SECOND_IN_DAY
mu = 3 * 10 ** (-3)
R = 700
d = 20

In [ ]:
s = (2 * np.pi * k * h * delta_P) / (q * mu) - np.log(2 * R / d)
s

3.993899374600457

In [ ]:
delta = np.log(2 * R / d) / (np.log(2 * R / d) + s)
delta

0.5154442901177416

In [ ]:
r_c = (d / 2) * np.exp (-s)
r_c

0.18427717267040944

Ответ:

$S = 3.99$

$\delta = 0.52$

$r_{C'} = 0.18\;см$